In [ ]:
# uncomment if running in vs code
# %cd ..
# %cd ..

## Kernel: stat_compound
Notebook is used to identify extremes and define the skew surge and precipitation marginals.

In [2]:
import xarray as xr
import hydromt
import pandas as pd
import numpy as np
import scipy.stats as sc
import matplotlib.pyplot as plt
from fitter import Fitter
%matplotlib inline

import Notebooks.Scripts.statistics_helper as stats_help

In [ ]:
df_comp = pd.read_csv('Data/all_drivers.csv')
df_comp = df_comp.set_index(pd.to_datetime(df_comp.iloc[:, 0]))
df_comp = df_comp.drop(columns = 'DateTime(UTC)')

df_comp.head()

In [ ]:
df_wl_ss = df_comp.copy()
df_wl_ss.drop(columns = [df_comp.columns[0], df_comp.columns[1], df_comp.columns[2], df_comp.columns[4], df_comp.columns[5]], inplace = True)
df_wl_ss.head()

In [ ]:
tidal_peaks = pd.read_csv('Data/skew_surge_tides.csv', parse_dates = ['DateTime(UTC)'])
tidal_peaks.set_index('DateTime(UTC)', inplace = True)
tidal_peaks.head()

In [6]:
cond_hh = tidal_peaks[tidal_peaks['Type'] == 'HH']
df_ss = cond_hh.iloc[:, 0]

## Threshold Selction
The purpose of doing a peak over threshold analysis is to not only have more representative extremes but also to ensure i.i.d. draws (block maxima only retrieves the maxima within its block, if this year was relatively calm, an non extreme event may be retrieved. Also, since there is no declustering time, two extremes could be correlated to one another if they occur at the end of a block and the start of the next consecutive block). A brute force approach will be used to ensure an extreme rate of at least one: Starting at the highest threshold value identified by the mean residual life and parameter stability plot will be slowly decreased until > 1.0 peaks per year on average are obtained. To be conservative, the declustering time was set to two weeks.

AIC vs AICc vs BIC. BIC is a baysian approach which prevents overfitting. Benefitial when there is little data.

## Skew Surge with MMSL (use pyextremes python package)

In [ ]:
stats_help.POT_threshold_plots(df_ss)

In [ ]:
stats_help.threshold_stable(df_ss, 100, [0.3, 0.45])

In [ ]:
ss_extremes, ss_quantile, ss_threshold = stats_help.POT_brute_force(df_ss, 0.32, decluster=2*7) # assumes 1 tidal period per day, df_ss is every tidal window

In [10]:
np.savetxt('fitted_stats/extreme_rate.txt', [ss_extremes['extremes_rate'].values.item()])

In [ ]:
stats_help.plot_extremes(df_ss, ss_extremes,
                         threshold = ss_quantile, ev_type = 'POT')
plt.show()

In [ ]:
stats_help.plot_fits(df_ss, ss_extremes)

In [ ]:
ss_marginal = ss_extremes['peaks'].to_dataframe()['peaks'].dropna()
ss_marginal.index = ss_marginal.index + np.timedelta64(6, 'h') # correct because of definition of skew surge (trough - trough)
ss_marginal

In [14]:
window_size = 24*3 # 3 days

window_s = stats_help.find_largest_within_window('Skew_surge (m)', ss_marginal, df_wl_ss['Precipitation (mm/hr)'], 12, window_size)
window_s['time_Skew_surge (m)'] = window_s['time_Skew_surge (m)'] - np.timedelta64(6, 'h')

In [15]:
window_s.to_csv('fitted_stats/compound_given_ss.csv', index = False) # Identified historical compound events

## Save marginals for skew surge and precipitation

In [16]:
bi_func = window_s.iloc[:, 2].sort_values(ascending=True)

In [17]:
import scipy.stats
distributions = [name for name in dir(scipy.stats) if isinstance(getattr(scipy.stats, name), scipy.stats.rv_continuous) or isinstance(getattr(scipy.stats, name), scipy.stats.rv_discrete)]

sens_dist = []
ignore = ['vonmises', 'uniform', 'wrapcauchy', 'bradford', 'argus',
          'arcsine', 'ksone', 'triang', 'powerlaw', 'truncexpon',
          'skewcauchy', 'wald', 'truncweibull_min', 'gibrat',
          'trapz', 'trapezoid', 'kappa4', 'truncpareto', 'anglit'] # trunc does not extrapolate, others dont make sense to fit
for dist in distributions:
    if dist not in ignore:
        sens_dist.append(dist)

In [ ]:
precip_curves = Fitter(bi_func, distributions=sens_dist, bins = 15)
precip_curves.fit()

In [ ]:
plt.rcParams.update({'font.size': 16})
precip_curves.summary(method = 'bic')
plt.xlabel('Precipitation Magnitude [mm/hr]')
plt.ylabel('Density [-]')

The Precipitation fit performs well on the non extreme events, but underestimates the probability of the extreme events. Visually, the lowest ks_statistic does not represent the emperical distribution the best.

In [ ]:
p_mag_marg = precip_curves.get_best('bic')
p_mag_marg

In [21]:
stats_help.save_marginal('fitted_stats/precipitation.json', p_mag_marg)

In [ ]:
name_dis = ss_extremes['distribution'].data.item()
param_names = ss_extremes['dparams'].data
param_values = ss_extremes['parameters'].data

ss_dic = stats_help.create_nested_dict(name_dis, param_names, param_values)
ss_dic

In [23]:
stats_help.save_marginal('fitted_stats/skew_surge.json', ss_dic)